# LLM Optimization Modelling Experiment

In [ ]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Image
from IPython.display import Markdown

## 1. Define the problem description

In [ ]:
problem = '''Imagine you own a small shop, and you're trying to manage your employees' working hours and sales targets efficiently.
You have three kinds of goals overall: sales goals, employee hours goals and overtime goals.

Sales Goals:
You have a target of selling 5500 units.
But sometimes, you might sell fewer (negative deviation) or more (positive deviation) than this target.
On average, your full-time employees will sell 5 records per hour and your part-time employees will sell 2 records per hour.

Employee Hours:
You have both full-time and part-time employees.
Your full-time employees are expected to work a total of 800 hours, while part-time employees are expected to work 320 hours.
Similarly, like sales, there could be deviations in these working hours.

Overtime:
Additionally, your employees might end up working overtime.
You have a target of 100 hours of overtime for full-time employees.
You are not concerned about limiting the overtime of part-time employees.
Again, you might deviate from this goal.

Each of these deviations has a different priority for you.
P1 to P4 are priority factors for each deviation.
Your most important goal is to achieve your sales goal.
Second to that is the goal to not exceed 100 overtime hours for full-time employees.
Then, you next priority is to fully utilize your full-time and part time employees.
Your last goal with the lowest priority, you want to keep general overtime hours for both kinds of employees as low as possible.
In addition to the general priorities of you goals, you are twice as concerned about fully utilizing the full-time employee hours.
Finally, overtime hours of part-time employees have a relative cost of three times that of full-time employees.

Now, the goal here is to minimize the overall impact of these deviations, considering their priorities.
This means finding the best balance between sales targets, employee hours, and overtime, while keeping in mind which deviations are more critical for your business.'''

## 2. Generate the mathematical model

In [ ]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [ ]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Mathematical Optimization Model for Employee Management

**Parameters:**

* **S_target:** Target sales (units) = 5500
* **S_FTH:** Sales per hour for a full-time employee = 5 units/hour
* **S_PTH:** Sales per hour for a part-time employee = 2 units/hour
* **H_FT:** Target hours for full-time employees = 800 hours
* **H_PT:** Target hours for part-time employees = 320 hours
* **OT_FT_target:** Target overtime hours for full-time employees = 100 hours
* **P1:** Priority factor for sales deviation
* **P2:** Priority factor for full-time overtime deviation
* **P3:** Priority factor for full-time employee hour deviation
* **P4:** Priority factor for part-time employee hour deviation
* **P5:** Priority factor for general overtime deviation

**Decision Variables:**

* **S_dev:** Deviation from the sales target (units)
* **H_FT_dev:** Deviation from the target full-time employee hours (hours)
* **H_PT_dev:** Deviation from the target part-time employee hours (hours)
* **OT_FT_dev:** Deviation from the target full-time overtime hours (hours)
* **OT_FT:** Total full-time overtime hours (hours)
* **OT_PT:** Total part-time overtime hours (hours)

**Objective Function:**

Minimize the weighted sum of deviations:

```
Minimize: 
  P1 * |S_dev| + 
  P2 * |OT_FT_dev| + 
  2 * P3 * |H_FT_dev| + 
  P4 * |H_PT_dev| + 
  P5 * (OT_FT + 3 * OT_PT) 
```

**Constraints:**

1. **Sales Deviation:** 
   * S_dev = S_target - (S_FTH * (H_FT + H_FT_dev + OT_FT) + S_PTH * (H_PT + H_PT_dev + OT_PT))

2. **Full-time Overtime Deviation:** 
   * OT_FT_dev = OT_FT_target - OT_FT

3. **Non-negativity of Overtime:**
   * OT_FT >= 0
   * OT_PT >= 0 

**Explanation:**

* The objective function minimizes the weighted sum of deviations from the various goals. 
* The priorities (P1 to P5) reflect the relative importance of each deviation. 
* The constraint on sales deviation calculates the difference between the target sales and the actual sales based on employee hours and sales rates.
* The constraint on full-time overtime deviation calculates the difference between the targeted and actual overtime hours for full-time employees.
* Non-negativity constraints ensure that overtime hours are not negative.

This model allows you to find the optimal balance between sales goals, employee hours, and overtime while considering the priorities you have set for your business. By adjusting the priority factors, you can fine-tune the model to better reflect your specific needs and preferences. 


## 3. Generate the pyomo code

In [ ]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text
response2 = generative_multimodal_model.generate_content([prompt2])

In [ ]:
#Showing the code in a formatted way
Markdown(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with your actual data
S_target = 5500  
S_FTH = 5  
S_PTH = 2  
H_FT = 800  
H_PT = 320 
OT_FT_target = 100  
P1 = 1  
P2 = 0.8  
P3 = 0.5 
P4 = 0.6 
P5 = 0.2  

# Model Creation
model = pyo.ConcreteModel()

# Decision Variables
model.S_dev = pyo.Var(within=pyo.Reals)
model.H_FT_dev = pyo.Var(within=pyo.Reals)
model.H_PT_dev = pyo.Var(within=pyo.Reals)
model.OT_FT_dev = pyo.Var(within=pyo.Reals)
model.OT_FT = pyo.Var(within=pyo.NonNegativeReals)
model.OT_PT = pyo.Var(within=pyo.NonNegativeReals)

# Objective Function
model.obj = pyo.Objective(
    expr= P1 * abs(model.S_dev) + 
          P2 * abs(model.OT_FT_dev) + 
          2 * P3 * abs(model.H_FT_dev) + 
          P4 * abs(model.H_PT_dev) + 
          P5 * (model.OT_FT + 3 * model.OT_PT), 
    sense=pyo.minimize)

# Constraints
model.sales_deviation = pyo.Constraint(
    expr= model.S_dev == S_target - (S_FTH * (H_FT + model.H_FT_dev + model.OT_FT) + S_PTH * (H_PT + model.H_PT_dev + model.OT_PT)))

model.ot_ft_deviation = pyo.Constraint(
    expr= model.OT_FT_dev == OT_FT_target - model.OT_FT)

# Solve the Model
solver = pyo.SolverFactory('cbc') # You can choose other solvers as well
results = solver.solve(model)

# Print Results
print(results)
print("Optimal Solution:")
print("Sales Deviation:", pyo.value(model.S_dev))
print("Full-Time Hours Deviation:", pyo.value(model.H_FT_dev))
print("Part-Time Hours Deviation:", pyo.value(model.H_PT_dev))
print("Full-Time Overtime Deviation:", pyo.value(model.OT_FT_dev))
print("Full-Time Overtime Hours:", pyo.value(model.OT_FT))
print("Part-Time Overtime Hours:", pyo.value(model.OT_PT))
```

This code defines the Pyomo model based on the mathematical formulation provided. I've included sample data for the parameters, which you should replace with your actual values. 

Remember that the choice of solver can impact the solution process. The 'cbc' solver is used here, but other solvers like 'glpk', 'gurobi', etc., can be used depending on your needs and availability. 


## 4. Input problem data and try running the generated code

In [3]:
import pyomo.environ as pyo

# Sample Data - Replace with your actual data
S_target = 5500
S_FTH = 5
S_PTH = 2
H_FT = 800
H_PT = 320
OT_FT_target = 100
P1 = 1
P2 = 0.8
P3 = 0.5
P4 = 0.6
P5 = 0.2

# Model Creation
model = pyo.ConcreteModel()

# Decision Variables
model.S_dev = pyo.Var(within=pyo.Reals)
model.H_FT_dev = pyo.Var(within=pyo.Reals)
model.H_PT_dev = pyo.Var(within=pyo.Reals)
model.OT_FT_dev = pyo.Var(within=pyo.Reals)
model.OT_FT = pyo.Var(within=pyo.NonNegativeReals)
model.OT_PT = pyo.Var(within=pyo.NonNegativeReals)

# Objective Function
model.obj = pyo.Objective(
    expr= P1 * abs(model.S_dev) +
          P2 * abs(model.OT_FT_dev) +
          2 * P3 * abs(model.H_FT_dev) +
          P4 * abs(model.H_PT_dev) +
          P5 * (model.OT_FT + 3 * model.OT_PT),
    sense=pyo.minimize)

# Constraints
model.sales_deviation = pyo.Constraint(
    expr= model.S_dev == S_target - (S_FTH * (H_FT + model.H_FT_dev + model.OT_FT) + S_PTH * (H_PT + model.H_PT_dev + model.OT_PT)))

model.ot_ft_deviation = pyo.Constraint(
    expr= model.OT_FT_dev == OT_FT_target - model.OT_FT)

# Solve the Model
solver = pyo.SolverFactory('couenne') # You can choose other solvers as well
results = solver.solve(model)

# Print Results
print(results)
print("Optimal Solution:")
print("Sales Deviation:", pyo.value(model.S_dev))
print("Full-Time Hours Deviation:", pyo.value(model.H_FT_dev))
print("Part-Time Hours Deviation:", pyo.value(model.H_PT_dev))
print("Full-Time Overtime Deviation:", pyo.value(model.OT_FT_dev))
print("Full-Time Overtime Hours:", pyo.value(model.OT_FT))
print("Part-Time Overtime Hours:", pyo.value(model.OT_PT))


Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 0
  Number of variables: 6
  Sense: unknown
Solver: 
- Status: ok
  Message: couenne\x3a Optimal
  Termination condition: optimal
  Id: 3
  Error rc: 0
  Time: 11.150506258010864
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Optimal Solution:
Sales Deviation: 0.0
Full-Time Hours Deviation: 0.0
Part-Time Hours Deviation: 0.0
Full-Time Overtime Deviation: -72.0
Full-Time Overtime Hours: 172.0
Part-Time Overtime Hours: 0.0


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [ ]:
print(response.text)

## Mathematical Optimization Model for Employee Management

**Parameters:**

* **S_target:** Target sales (units) = 5500
* **S_FTH:** Sales per hour for a full-time employee = 5 units/hour
* **S_PTH:** Sales per hour for a part-time employee = 2 units/hour
* **H_FT:** Target hours for full-time employees = 800 hours
* **H_PT:** Target hours for part-time employees = 320 hours
* **OT_FT_target:** Target overtime hours for full-time employees = 100 hours
* **P1:** Priority factor for sales deviation
* **P2:** Priority factor for full-time overtime deviation
* **P3:** Priority factor for full-time employee hour deviation
* **P4:** Priority factor for part-time employee hour deviation
* **P5:** Priority factor for general overtime deviation

**Decision Variables:**

* **S_dev:** Deviation from the sales target (units)
* **H_FT_dev:** Deviation from the target full-time employee hours (hours)
* **H_PT_dev:** Deviation from the target part-time employee hours (hours)
* **OT_FT_dev:** Deviati

In [ ]:
print(response2.text)

```python
import pyomo.environ as pyo

# Sample Data - Replace with your actual data
S_target = 5500  
S_FTH = 5  
S_PTH = 2  
H_FT = 800  
H_PT = 320 
OT_FT_target = 100  
P1 = 1  
P2 = 0.8  
P3 = 0.5 
P4 = 0.6 
P5 = 0.2  

# Model Creation
model = pyo.ConcreteModel()

# Decision Variables
model.S_dev = pyo.Var(within=pyo.Reals)
model.H_FT_dev = pyo.Var(within=pyo.Reals)
model.H_PT_dev = pyo.Var(within=pyo.Reals)
model.OT_FT_dev = pyo.Var(within=pyo.Reals)
model.OT_FT = pyo.Var(within=pyo.NonNegativeReals)
model.OT_PT = pyo.Var(within=pyo.NonNegativeReals)

# Objective Function
model.obj = pyo.Objective(
    expr= P1 * abs(model.S_dev) + 
          P2 * abs(model.OT_FT_dev) + 
          2 * P3 * abs(model.H_FT_dev) + 
          P4 * abs(model.H_PT_dev) + 
          P5 * (model.OT_FT + 3 * model.OT_PT), 
    sense=pyo.minimize)

# Constraints
model.sales_deviation = pyo.Constraint(
    expr= model.S_dev == S_target - (S_FTH * (H_FT + model.H_FT_dev + model.OT_FT) + S_PTH * (H_PT + mode